In [1]:
import pandas as pd
import re

In [2]:
data=pd.read_csv('train.csv', encoding = "ISO-8859-1")

In [3]:
import re 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def preProcessString(text):
    #Strips quotes at end of text
    text=text.strip('')
    #Rmoving twitter handles @user
    text=re.sub("@[\w]*"," ",text) 
    # Remove URLs with the space
    text = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' ', text)
    # Strip space, " and ' from text
    text = text.strip(' "\'')
    #Remove repeated letters of string such as jusssssst to just 
    text=re.sub(r'(.)\1{3,}', r'\1', text)
    # remove all special characters
    text = re.sub('[^A-Za-z]', ' ', text)
    #replace two or more dots with space
    text = re.sub("\\.{2,}"," ",text);
    # converting all text into small letters and store them as words for further processing
    text_list = text.lower().split()
     # stemming the words (removing prefix and postfix) using Porter stemming algorithm..
    text_list = [ps.stem(word) for word in text_list]
    return ' '.join(text_list)
    

In [4]:
data['Preprocessed_data']=data['SentimentText'].apply(preProcessString)

In [5]:
data.head()

,ItemID,Sentiment,SentimentText,Preprocessed_data
0,1,0,is so sad for my APL frie...,is so sad for my apl friend
1,2,0,I missed the New Moon trail...,i miss the new moon trailer
2,3,1,omg its already 7:30 :O,omg it alreadi o
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...,omgaga im sooo im gunna cri i ve been at thi d...
4,5,0,i think mi bf is cheating on me!!! ...,i think mi bf is cheat on me t t


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
Count_vectorization=CountVectorizer()
tf_idf_transform = TfidfTransformer()

In [7]:
train, test = train_test_split(data, test_size=0.20)

In [8]:
train.head()

,ItemID,Sentiment,SentimentText,Preprocessed_data
91778,91790,1,@ColetteA I like your picture,i like your pictur
61636,61648,0,@bigassbadger hope @sapphyno1 hasnt ditched yo...,hope hasnt ditch you live on twitter
10361,10373,1,&quot;Peter your dog is giving me diabetes&quot;,quot peter your dog is give me diabet quot
56973,56985,1,@barrysma NEW motorcycle and you POPPED a cabl...,new motorcycl and you pop a cabl alreadi wow y...
27695,27707,1,"@ahj Should be doing coursework, but that vide...",should be do coursework but that video is awes...


In [9]:
#Count_vectorization=CountVectorizer(ngram_range = (1, 2))

X_train_count_vect = Count_vectorization.fit_transform(train["Preprocessed_data"])
X_train_tf_idf_vector = tf_idf_transform.fit_transform(X_train_count_vect)

X_test_count_vect = Count_vectorization.transform(test["Preprocessed_data"])
X_test_tf_idf_vector = tf_idf_transform.transform(X_test_count_vect)

y_train = train['Sentiment']

y_test = test['Sentiment']


In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model

predict_Text=dict()

In [11]:
model = MultinomialNB().fit(X_train_tf_idf_vector, y_train)
predict_Text['Multinomial'] = model.predict(X_test_tf_idf_vector)

model = BernoulliNB().fit(X_train_tf_idf_vector, y_train)
predict_Text['Bernoulli'] = model.predict(X_test_tf_idf_vector)



lg1 = LogisticRegression(C=1)
lgReg_result = lg1.fit(X_train_tf_idf_vector, y_train)
predict_Text['Logistic'] = lgReg_result.predict(X_test_tf_idf_vector)

Dc_Tree_model=DecisionTreeClassifier()
decisionTree_result=Dc_Tree_model.fit(X_train_tf_idf_vector, y_train)
predict_Text['DecisionTree'] = decisionTree_result.predict(X_test_tf_idf_vector)

clf = linear_model.SGDClassifier(loss='log',  max_iter=10000)
clf.fit(X_train_tf_idf_vector, y_train)
predict_Text['SVM SGD Classifier'] = clf.predict(X_test_tf_idf_vector)

C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
print("Logistic Classification Report")
print(metrics.classification_report(y_test, predict_Text['Logistic'], target_names = ["Negative","Positive"]))
print("Accuracy  %0.3f%%" %(accuracy_score(y_test,predict_Text['Logistic'])*100))
print('==================================================================================================')
print("Bernoulli Naive Bayesian Classification Report")
print(metrics.classification_report(y_test, predict_Text['Bernoulli'], target_names = ["Negative","Positive"]))
print("Accuracy %0.3f%%" %(accuracy_score(y_test,predict_Text['Bernoulli'])*100))
print('==================================================================================================')
print("Mulitnomial Naive Bayesian Classification Report")
print(metrics.classification_report(y_test, predict_Text['Multinomial'], target_names = ["Negative","Positive"]))
print("Accuracy  %0.3f%%" %(accuracy_score(y_test,predict_Text['Multinomial'])*100))
print('==================================================================================================')
print("Decision Tree Classification Report")
print(metrics.classification_report(y_test, predict_Text['DecisionTree'], target_names = ["Negative","Positive"]))
print("Accuracy  %0.3f%%" %(accuracy_score(y_test,predict_Text['DecisionTree'])*100))
print('==================================================================================================')
print("SVM SGD Classifier")
print(metrics.classification_report(y_test, predict_Text['SVM SGD Classifier'], target_names = ["Negative","Positive"]))
print("Accuracy  %0.3f%%" %(accuracy_score(y_test,predict_Text['SVM SGD Classifier'])*100))

Logistic Classification Report
              precision    recall  f1-score   support

    Negative       0.76      0.69      0.73      8646
    Positive       0.78      0.84      0.81     11352

   micro avg       0.77      0.77      0.77     19998
   macro avg       0.77      0.76      0.77     19998
weighted avg       0.77      0.77      0.77     19998

Accuracy  77.403%
Bernoulli Naive Bayesian Classification Report
              precision    recall  f1-score   support

    Negative       0.74      0.66      0.70      8646
    Positive       0.76      0.82      0.79     11352

   micro avg       0.75      0.75      0.75     19998
   macro avg       0.75      0.74      0.74     19998
weighted avg       0.75      0.75      0.75     19998

Accuracy 75.268%
Mulitnomial Naive Bayesian Classification Report
              precision    recall  f1-score   support

    Negative       0.78      0.59      0.67      8646
    Positive       0.74      0.88      0.80     11352

   micro avg       0